# NPC AI: Advanced Neuro-Symbolic Research Pipeline
**Author:** Le Tran Minh Phuc

This notebook implements the **State-of-the-Art (SOTA) Neuro-Symbolic Architecture** for NPC logic. Unlike basic RAG comparisons, this pipeline implements:
1.  **Dynamic Knowledge Graph Construction**: Using LLM-based Open Information Extraction (OIE) to build a graph from raw lore text at runtime.
2.  **Fine-Tuning on PersonaChat**: Using the `proj-persona/PersonaChat` dataset for high-fidelity roleplay adaptation.
3.  **Advanced RAG**: Recursive Semantic Chunking for optimal context retrieval.
4.  **Robust Training**: Includes Checkpoint Resume logic for long runs.

### Hardware Requirements:
- **Accelerator**: GPU T4 x2 (Turn on in Kaggle Settings)


In [ ]:
# === SECTION 1: SETUP & INSTALLATION ===
# Install Unsloth, Llama-cpp, and Advanced NLP Tools
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -q llama-cpp-python
!pip install -q sentence-transformers faiss-cpu networkx bert_score huggingface_hub pandas psutil langchain-text-splitters datasets

In [ ]:
# === SECTION 2: FINE-TUNING ON REAL DATASET (PersonaChat) ===
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

max_seq_length = 2048
dtype = None 
load_in_4bit = True

print("Loading Phi-3 Mini...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("Adding LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

In [ ]:
# 2.1 Load and Format PersonaChat Dataset
# Transforming PersonaChat (Dialogue) into Instruction Format for NPC training
dataset = load_dataset("proj-persona/PersonaChat", split="train[:5000]") # subset for speed

alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are an NPC with these traits: {}. \nUser says: "{}"

### Response:
{}"""

def format_persona_chat(examples):
    personalities = examples['personality']
    histories = examples['history']
    candidates = examples['candidates']
    
    texts = []
    for i in range(len(personalities)):
        # Extract last turn 
        user_input = histories[i][-1] if histories[i] else "Hello"
        npc_response = candidates[i][-1] # Gold response
        persona_desc = "; ".join(personalities[i])
        
        text = alpaca_prompt.format(persona_desc, user_input, npc_response) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts }

print("Formatting PersonaChat Dataset...")
dataset = dataset.map(format_persona_chat, batched = True)
print(f"Training on {len(dataset)} real dialogue examples.")

In [ ]:
# 2.2 Train with Checkpoint & Resume Logic
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.trainer_utils import get_last_checkpoint
import os

output_dir = "outputs"

# Resume Checkpoint Logic
last_checkpoint = None
if os.path.isdir(output_dir):
    last_checkpoint = get_last_checkpoint(output_dir)
    if last_checkpoint:
        print(f"Found checkpoint: {last_checkpoint}. Resuming training...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100, 
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        # Checkpoint Settings
        save_strategy = "steps",
        save_steps = 20, # Save every 20 steps
        save_total_limit = 2, # Keep only last 2 checkpoints
    ),
)

print("Starting Fine-Tuning on PersonaChat...")
trainer.train(resume_from_checkpoint=last_checkpoint)

# Save trained model
model.save_pretrained_gguf("model_finetuned", tokenizer, quantization_method = "q4_k_m")
trained_model_path = "model_finetuned/unsloth.Q4_K_M.gguf"

In [ ]:
# === SECTION 3: ADVANCED NEURO-SYMBOLIC INFERENCE ===
# NOTE: We import the pipeline from the repository to ensure reproducibility
import sys
import os

# 1. Setup Repo Access
if not os.path.exists("NPC-AI"):
    print("Cloning repository...")
    !git clone https://github.com/minhphuc477/NPC-AI.git
else:
    print("Repository already exists.")

# 2. Add to Path
sys.path.append(os.path.abspath("NPC-AI"))

# 3. Import from Core
try:
    from core.neuro_symbolic_pipeline import NeuroSymbolicPipeline
    print("Successfully imported NeuroSymbolicPipeline from repo!")
except ImportError as e:
    print(f"Error importing from repo: {e}")
    print("Ensure you are running this in a Kaggle environment where the repo is cloned or mounted.")

In [ ]:
# === SECTION 4: SCIENTIFIC BENCHMARK ===

# Using the sophisticated engine
print("Initializing Neuro-Symbolic Engine...")
# Use the trained model from previous step
engine = NeuroSymbolicPipeline(trained_model_path)

configs = {
    "Neuro-Symbolic (Full)": {"enable_rag": True, "enable_graph": True},
    "RAG Only": {"enable_rag": True, "enable_graph": False},
    "Graph Only": {"enable_rag": False, "enable_graph": True}
}

prompts = [
    "Where can I find finding the Elder Stone?", 
    "Why did Duke Varen betray the King?",
    "How do I make a healing potion?"
]

results = []
print("Running Experiments...")

for name, cfg in configs.items():
    for p in prompts:
        res = engine.generate(p, cfg)
        results.append({
            "Config": name,
            "Prompt": p,
            "Latency": res['latency_ms'],
            "Throughput": res['tps'],
            "Response": res['text'][:50] + "..."
        })

import pandas as pd
df = pd.DataFrame(results)
print(df.groupby("Config")[["Latency", "Throughput"]].mean().to_markdown())
df.to_csv("final_results.csv")